In [26]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score, roc_curve, auc
import lightgbm as lgb
# 导入训练数据
data = pd.read_csv('../yjf_2_data/1003531_user_personas_2020-10-20.csv')
print(len(list(set(list(data['platform_open_id'])))))
data = data.drop(['suc_goods_num_d721','suc_goods_num_d720', 'suc_goods_num_d360', 'suc_goods_num_d90', 'suc_order_cnt', 
          'suc_order_cnt_d720', 'suc_order_cnt_d360', 'suc_order_cnt_d90','platform_open_id','time_span'], axis=1)
data['label'] = data['suc_order_cnt_d721']
data = data.drop('suc_order_cnt_d721',axis=1)
# 弃去缺失值
data.dropna(inplace=True)
print(f'-------一共有{len(data)}条数据')
data.head()

1000000
-------一共有1000000条数据


,is_member,member_grade,province_level,city_level,register_time,amount,suc_payment_d720,suc_payment_d360,suc_payment_d90,pre_order_pay_d721,...,pre_order_pay_d90,last_pay_at,last_payment,last_goods_num,first_pay_at,first_payment,first_goods_num,join_num_time_90d,join_num_time_180d,label
0,0,0,2,1,9,0,0,0,0,0,...,0,9,0,0,9,0,0,0,0,0
1,0,0,2,1,9,9,9,9,0,8,...,0,7,1,1,7,9,1,0,0,2
2,1,1,0,0,7,0,0,0,0,0,...,0,9,0,0,9,0,0,0,0,0
3,0,0,3,2,9,12,9,9,0,9,...,0,7,9,1,8,9,1,0,0,2
4,0,0,2,1,9,12,12,12,0,8,...,0,6,9,1,7,1,1,0,0,3


In [2]:
good_num = data.groupby('label')['label'].count()
print(f'----购买次数的分布----：{good_num}')
print(sum(data['label']==1)/float(len(data)))

----购买次数的分布----：label
0    235488
1    596867
2    131678
3     26197
4      9659
5        84
6        27
Name: label, dtype: int64
0.596867


In [3]:
# data['label'] = data.label.apply(lambda x: 1 if x==1 else 0)
# data

In [21]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
def get_train_testDF(train_df,label_df):
    skv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    trainX = []
    trainY = []
    testX = []
    testY = []
    for train_index, test_index in skv.split(X=train_df, y=label_df):
        train_x, train_y, test_x, test_y = train_df.iloc[train_index, :], label_df.iloc[train_index], \
                                            train_df.iloc[test_index, :], label_df.iloc[test_index]

        trainX.append(train_x)
        trainY.append(train_y)
        testX.append(test_x)
        testY.append(test_y)
    return trainX, testX, trainY, testY


train_X, train_y = data.drop(['label'], axis=1), data['label']
X_train, X_valid, y_train, y_valid = get_train_testDF(train_X, train_y)
y_train[0].values

array([0, 2, 0, ..., 1, 1, 1], dtype=int64)

In [25]:
clf = lgb.LGBMClassifier(
            # **params
            objective='multiclass',
            random_state=626,
            n_estimators=800,
            learning_rate=0.1,
            max_depth=-1,
            num_leaves=127,
            colsample_bytree=0.8,
            subsample=0.8,
            reg_alpha=0.1,
            reg_lambda=0.2
        )
clf.fit(X_train[0].values, y_train[0].values)
pred_label = clf.predict(X_valid[0].values)

In [24]:
result = classification_report(y_valid[0].values, pred_label, digits=3, output_dict=True)
print(result)

C:\Users\Administrator\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 47098}, '1': {'precision': 0.9864750843756204, 'recall': 0.99899474755598, 'f1-score': 0.9926954437049709, 'support': 119373}, '2': {'precision': 0.9539820932869375, 'recall': 0.9264884568651276, 'f1-score': 0.9400342881359197, 'support': 26336}, '3': {'precision': 0.7886246122026888, 'recall': 0.727810650887574, 'f1-score': 0.756998213222156, 'support': 5239}, '4': {'precision': 0.6772983114446529, 'recall': 0.5605590062111802, 'f1-score': 0.6134239592183517, 'support': 1932}, '5': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 16}, '6': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 6}, 'accuracy': 0.978235, 'macro avg': {'precision': 0.6294828716156999, 'recall': 0.6019789802171231, 'f1-score': 0.6147359863259141, 'support': 200000}, 'weighted avg': {'precision': 0.9771035366850834, 'recall': 0.978235, 'f1-score': 0.977534124710109, 'support': 200000}}
